In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from getpass import getpass
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate
from langchain import hub
from langchain_openai import ChatOpenAI
from graphdatascience import GraphDataScience
import pandas as pd

/Users/nathansmith/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
openai_api_key = getpass()

 ········


In [4]:
neo4j_password = getpass()

 ········


In [5]:
neo4j_uri = "neo4j+s://2fe3bf28.databases.neo4j.io"
neo4j_user = "neo4j"

In [6]:
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [1]:
def query_movies(query_string, query_type):
    queries = {
        "movie": """
                    WITH genai.vector.encode($queryString, "OpenAI", {token:$token, model: "text-embedding-3-small"}) as v
                    CALL db.index.vector.queryNodes("movie_text_vectors", 50, v) YIELD node, score 
                    RETURN score, node.title AS title, node.overview AS overview
                    ORDER BY score DESC""",
        "theme": """
                    WITH genai.vector.encode($queryString, "OpenAI", {token:$token, model: "text-embedding-3-small"}) as v
                    CALL db.index.vector.queryNodes("theme_vectors", 25, v) YIELD node, score 
                    MATCH (node)<-[:HAS_THEME]-(m)
                    RETURN node.description AS theme, score, m.title AS title, m.overview AS overview
                    ORDER BY score DESC
                    LIMIT 50
                    """,
        "theme_group": """
                    WITH genai.vector.encode($queryString, "OpenAI", {token:$token, model: "text-embedding-3-small"}) as v
                    CALL db.index.vector.queryNodes("theme_group_summary_vectors", 25, v) YIELD node, score 
                    MATCH (node)<-[:IN_GROUP]-()<-[:HAS_THEME]-(m)
                    RETURN node.descriptions AS themes, score, m.title AS title, m.overview AS overview
                    ORDER BY score DESC
                    LIMIT 50"""
    }
        
    movie_df = gds.run_cypher(queries[query_type], {"queryString": query_string, "token":openai_api_key})
    return movie_df

In [286]:
theme_group_query = """
MATCH (node)<-[:IN_GROUP]-()<-[:HAS_THEME]-(m)
WITH m, max(score) AS score, collect(node.descriptions)[0] AS themes 
RETURN m.overview AS text, score, m{.title, themes:themes} AS metadata
ORDER BY score DESC 
LIMIT 60
"""

theme_query = """
MATCH (node)<-[:HAS_THEME]-(m)
WITH m, max(score) AS score 
RETURN m.overview AS text, score, m{.title, .url} AS metadata
ORDER BY score DESC
limit 60
"""

movie_query = """
MATCH (node)
RETURN node.overview AS text, score, node{.title, .url} AS metadata
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a movie expert providing information about movies.
                  Be as helpful as possible and return as much information as possible.
                  Do not answer any questions that do not relate to movies.
                  Provide your answer as a pipe-separated list of movie titles.
                  Answer any questions based solely on the context below:
                  <context>
                  {context}
                  </context>"""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("user", "{input}"),
])

movie_info_template= "overview: {page_content}, title: {title}"
document_prompt = PromptTemplate(input_variables=["page_content", "title"],
                                 template=movie_info_template
                                 )

combine_docs_chain = create_stuff_documents_chain(
    ChatOpenAI(temperature=0, openai_api_key=openai_api_key), prompt, document_prompt=document_prompt)


def make_chain(query, index, k):
    vectorstore = Neo4jVector.from_existing_index(
        OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key),
        index_name=index,
        retrieval_query=query,
        url=neo4j_uri,
        username=neo4j_user,
        password=neo4j_password)
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    chain = create_retrieval_chain(retriever, combine_docs_chain)
    return chain

movie_chain = make_chain(movie_query, "movie_text_vectors", 60)
theme_chain = make_chain(theme_query, "theme_vectors", 10)
theme_group_chain = make_chain(theme_group_query, "theme_group_summary_vectors", 5)


In [282]:
def get_selected_movies(response, query, retriever):
    context = response['context']
    answers = response['answer'].split(", ")
    movie_list = []
    for answer in answers:
        matched_context = [{"title":answer, "overview": doc.page_content}
                           for doc in context if doc.metadata['title'] == answer]
        if len(matched_context) == 1:
            movie_list.append(matched_context[0])
        else:
            movie_list.append({"title":answer})
    selected_movie_df = pd.DataFrame(movie_list)
    selected_movie_df['query'] = query
    selected_movie_df['retriever'] = retriever
    return selected_movie_df

def compare_searches(query):
    movie_result = movie_chain.invoke({"input": query})
    theme_result = theme_chain.invoke({"input": query})
    theme_group_result = theme_group_chain.invoke({"input": query})
    
    movie_selected = get_selected_movies(movie_result, query, "movie")
    theme_selected = get_selected_movies(theme_result, query, "theme")
    theme_group_selected = get_selected_movies(theme_group_result, query, "theme_group")
    selected_df = pd.concat([movie_selected, theme_selected, theme_group_selected])
    
    movie_set = set([document.metadata['title'] for document in movie_result['context']])
    theme_set = set([document.metadata['title'] for document in theme_result['context']])
    theme_group_set = set([document.metadata['title'] for document in theme_group_result['context']])

    movie_only = movie_set.difference(theme_set.union(theme_group_set))
    theme_only = theme_set.difference(movie_set.union(theme_group_set))
    theme_group_only = theme_group_set.difference(movie_set.union(theme_set))
    movie_theme = movie_set.intersection(theme_set).difference(theme_group_set)
    movie_theme_group = movie_set.intersection(theme_group_set).difference(theme_set)
    theme_theme_group = theme_set.intersection(theme_group_set).difference(movie_set)
    all_lists = movie_set.intersection(theme_set).intersection(theme_group_set)
    
    output_df = pd.DataFrame()
    output_df.loc["movie_answer", "result"] = movie_result['answer']
    output_df.loc["theme_answer", "result"] = theme_result['answer']
    output_df.loc["theme_group_answer", "result"] = theme_group_result['answer']
    output_df.loc["movie_only_context", "result"] = movie_only
    output_df.loc["theme_only_context", "result"] = theme_only
    output_df.loc["theme_group_only_context", "result"] = theme_group_only
    output_df.loc["movie_theme_context", "result"] = movie_theme
    output_df.loc["movie_theme_group_context", "result"] = movie_theme_group
    output_df.loc["theme_theme_group_context", "result"] = theme_theme_group
    output_df.loc["all_context", "result"] = all_lists
    
    return output_df, selected_df

In [57]:
pd.set_option('display.max_colwidth', None)

In [277]:
output_df, current_selected_df = compare_searches("What are some titles of movies about classical music?")
selected_df = current_selected_df
output_df

,result
movie_answer,Pioniere der Filmmusik - Europas Sound für Hollywood | Lenechka | Symphonic Sketches | A Piano Tune | Symphony of Suffering | The Piper | IN THE BASEMENT OF APOLLO HALL | Sergey Slonimsky. Bells | Cici's Sonata | Passing Time
theme_answer,Symphony n. 9 by Ludwig van Beethoven in St. Mark’s Square | Als ik zelf de zon ben | Symphonic Sketches | Crescendo | Pioniere der Filmmusik - Europas Sound für Hollywood | Beautiful Pleasant Trivial | Double Around the Interlude
theme_group_answer,Symphony n. 9 by Ludwig van Beethoven in St. Mark’s Square | Als ik zelf de zon ben | Symphonic Sketches | Crescendo | Mozart's Requiem | The Only Girl in the Orchestra | The Orchestra Chuck Built | Celtic Woman: 20th Anniversary Show | My Favorite Things: The Rodgers & Hammerstein 80th Anniversary Concert | Música! | Do You See Me? | Eye for an Eye | Femboy Beat Making: The Making of Trout Jazz | Bee's Knee's | Le Nozze Di Figaro - Opera Ballet Vlaanderen | Horrible Histories: 'Orrible Opera | Das Märchen von der Zauberflöte | Goddess of Speed | The Runaway | My Name's Ben Folds – I Play Piano | Fantasymphony II - A Concert of Fire and Magic | Nosferatu: A Symphony of Horror | American Symphony | I Hear a Symphony | Doraemon the Movie: Nobita's Earth Symphony
movie_only_context,"{Out of Jungle, Dancing Fruit, Drift, Tempo, Greetings from Crîngași, Fuego Sagrado, Sergey Slonimsky. Bells, Casanova and The Three Graces, Poster Child, Meshes, IN THE BASEMENT OF APOLLO HALL, Chamariz, The Battle for Swan Lake, Cici's Sonata, Asfalto che suona, In The Shadow Of The Horns, The Cigarette, Vampire Stories: Bluttanz, My Fear in My Arms, Pleiades (or Going Home), Moro, Ladan, The More I Zoom in on the Image of These Dogs, The Clearer it Becomes That They Are Related to the Stars., Shards of Moon, Film Fest Gent, A Love Letter to Cinema, Untitled, The Piper, The Swell, Songs from the Hole, fly, Sanctuary, A Piano Tune, Symphony of Suffering, In the Conservatory, Milestones, Istanbul Trilogy: Music, A Slippage in Five Movements, Passing Time, Lenechka, Music Genres. Voices of the World, The Musical Mind: A Portrait in Process, echo}"
theme_only_context,"{Angelheaded Hipster: The Songs of Marc Bolan & T. Rex, Double Around the Interlude, We'll be Singing by September, Vika!, The Tragic Death of a Frenchman, Pretty Ugly - The Story Of The Lunachicks, Blockbuster}"
theme_group_only_context,"{Celtic Woman: 20th Anniversary Show, Música!, Femboy Beat Making: The Making of Trout Jazz, Eye for an Eye, My Name's Ben Folds – I Play Piano, Do You See Me?, Mozart's Requiem, Bee's Knee's, American Symphony, The Orchestra Chuck Built, My Favorite Things: The Rodgers & Hammerstein 80th Anniversary Concert, The Runaway, Le Nozze Di Figaro - Opera Ballet Vlaanderen, Between the Temples, Das Märchen von der Zauberflöte, The Only Girl in the Orchestra, Stars, The Devil Went Down To Islington, Horrible Histories: 'Orrible Opera, Goddess of Speed}"
movie_theme_context,"{Beautiful Pleasant Trivial, Pioniere der Filmmusik - Europas Sound für Hollywood}"
movie_theme_group_context,"{I Hear a Symphony, Nosferatu: A Symphony of Horror, Fantasymphony II - A Concert of Fire and Magic, Doraemon the Movie: Nobita's Earth Symphony}"
theme_theme_group_context,"{Crescendo, Symphony n. 9 by Ludwig van Beethoven in St. Mark’s Square, Als ik zelf de zon ben}"
all_context,{Symphonic Sketches}


In [265]:
output_df, current_selected_df = compare_searches("What are some titles of movies about baseball?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,"Thunder Rolls! The World of Blind Baseball, Indoor Baseball, unDEFILED, Below the Rim, Carson Bigbee: The Pirate of America's Pastime, The Hurler: A Campion's Tale, Sole: The Movie"
theme_answer,"The Astros Edge, Shame, Smashing Pumpkins, Swimming in a Sand Pool, Consumed, Indoor Baseball, Thunder Rolls! The World of Blind Baseball"
theme_group_answer,"Carson Bigbee: The Pirate of America's Pastime, unDEFILED, Monster Under The River, Heritage Minutes: Mary ""Bonnie"" Baker, Samurai Japan: The Story Behind the WBC Clean Sweep, The Astros Edge, Shame"
movie_only_context,"{Movie (Choke), Falling in Love Like in Movies, How India Stole and Killed Cricket, Ethan the Roid Junkie, The Power of the Strike, Brothers, Groundbreakers, Below the Rim, BRADMAN: The Movie, Bowlhead, The Movie Man, Holiday Heist, Labranza, Squatch Junkies, Frank Capra: Mr. America, Young Again, The Big Catch, Blood Money, No Ball Games, Blue Star, Swing Bout, Humbly Super Famous, Shohei Ohtani: Beyond the Dream, Joe, The Hurler: A Campion's Tale, Left Brain, A Movie With No Views On Letterboxd, Sole: The Movie, Because They Believed, Butchy's Big Pitch, Kanashimi, Fuggeddaboutitt, Why Movies?, Heads and Tails, Made in Massachusetts: 100 Years of Filmmaking in the Bay State, Diamante – Fussballgott, I'll Crush Y'all, The Godfrey Project}"
theme_only_context,"{Trouble with Johnny, The Tragic Death of a Frenchman, Consumed, Swimming in a Sand Pool}"
theme_group_only_context,"{The Many Worlds of George Goodman, Divine Providence, Al Chile!, Sue Bird: In the Clutch, 5th QTR, One More Shot, Charles Barkley Comes Out, After Dawn, Wardell Stephen Curry, Out of Bounds, Heritage Minutes: Mary ""Bonnie"" Baker, The Ticket, Big Time, Giannis: The Marvelous Journey, Rudy Gobert N°27, Backboard, Leo, Pickup, Eureka, Hard Feelings, Hada, The Netflix Slam, Hoop Cities - NBA Feature Documentary, The Hoop Dreams Holiday Special, Djokovic Unmasked, Imbroda, el legado del maestro, Home and Away, Kingdome, Young Guns: The New Tennis Titans, La cancha, Rally, Dealova, Warrior Strong}"
movie_theme_context,"{Thunder Rolls! The World of Blind Baseball, Indoor Baseball}"
movie_theme_group_context,"{Lady Ballers, Iceman: A George Gervin Story, GROUNDED., Carson Bigbee: The Pirate of America's Pastime, Freaky Tales, Samurai Japan: The Story Behind the WBC Clean Sweep, Gold Blooded - NBA Feature Documentary, Monster Under The River, unDEFILED}"
theme_theme_group_context,"{Shame, Smashing Pumpkins}"
all_context,{The Astros Edge}


In [266]:
output_df, current_selected_df = compare_searches("What are some movie titles about freedom?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,"Liberty, Naked Nations – Tribe Hong Kong, American Outlaws, FOUR DAYS FOR FREEDOM: NAPLES 1943, If Everyone Just Leaves, The Conqueror (Hollywood Fallout), Trailer of a Film That Will Never Exist: Phony Wars, Celluloid Underground, Love of the Soviet Union, The Movie Man, Communist Osman, From 8 Movies To 1, Poor Things, Silent Cries, The Godfrey Project, Cine Libre, Why Movies?, Lines of Resistence, Nobody Leaves Alive, Fly Free, When We Return, What is the Point?, The Busy World Around Me, My Sister's Keeper, Exile, Che, Letters To Freedom, Infinity!, Showdown at the Grand, Memories, Air Force Two, Under a Blue Sun, Land of My Dreams, A Long Shot, Fly Hard, Prisoners, Waterbird, Landless, The Nice Trip, Exit, Happy Places, Kanaani, In Freedom, Kannagi"
theme_answer,"FREEDOM MAN: THE BALLAD OF THE BRICK, It's A Gray Gray World, Sabittri, The Ticket, Getting Fired From Office, Defend Your Name, Breaking the Ice, Our Land, Our Freedom, Decisions Decisions, Stamp Our Story, Code Break, My Freedom, Hunting Party, FOUR DAYS FOR FREEDOM: NAPLES 1943, Sorbeltza, Feet Fear Freedom, Rise, 4 Walls, Blue, Nyanga, Dealer, The Prince of Egypt: The Musical, The Prisoner of Wakhan, Amanita, Rencores, Ukrainian Identity, Freedom, Zindagi Zindabaad, Not Born Yesterday, Psychedelic Revolution: The Secret History of the LSD Trade, Gendelyk, Free Beats: The Musical Journey of CHEN Ming Chang, Girl, The History of Indian Elections, Transformation, Fille Fleur, Night in Newark, Boots On The Ground, Flowing - My Dream of More Freedom, El Grito De Emma, The Last Thread, No Way Through, Ex You, The Strike, Here and Nowhere, Please Look Up, Dream Kite, Bad Romance: The Vicky White Story"
theme_group_answer,"Bad Romance: The Vicky White Story, Hunting Party, Notes on a Summer, Breaking the Ice, Decisions Decisions, Stamp Our Story, Code Break, My Freedom, Rise, Metal Belt, Sorbeltza, FOUR DAYS FOR FREEDOM: NAPLES 1943, Zindagi Zindabaad, My Freedom, The Prisoner of Wakhan, Amanita, Rencores, Ukrainian Identity, Freedom, Boots On The Ground, Flowing - My Dream of More Freedom, No Way Through, Ex You, The Strike, Here and Nowhere, Beyond the Blue Border, Water under the bridge, Naked Nations – Tribe Hong Kong, Na Marei, Derwila, Getting Fired From Office"
movie_only_context,"{Movie (Choke), Nobody Leaves Alive, Silent Cries, Winner, Liberty, Celluloid Underground, Lines of Resistence, The Movie Man, Exit, Happy Places, Fly Hard, Love of the Soviet Union, Communist Osman, Kannagi, I Believe in America, Showdown at the Grand, Infinity!, Fly Free, PRODUCT, If Everyone Just Leaves, Landless, Little Death, Air Force Two, Prisoners, Poor Things, The Busy World Around Me., Cine Libre, Exile, Kanaani, Waterbird, My Sister's Keeper, From 8 Movies To 1, The Nice Trip, Memories, Under a Blue Sun, What is the Point?, Letters To Freedom, Why Movies?, When We Return, A Long Shot, The Conqueror (Hollywood Fallout), In Freedom, The Godfrey Project, Che, Trailer of a Film That Will Never Exist: Phony Wars, Land of My Dreams, American Outlaws}"
theme_only_context,"{FREEDOM MAN: THE BALLAD OF THE BRICK, The Ticket, Shyamchi Aai, Sabittri, Sedin Kuyasha Chilo, It's A Gray Gray World!}"
theme_group_only_context,"{Beyond the Blue Border, Na Marei, Water under the bridge, Outside It's Burning, Derwila}"
movie_theme_context,{}
movie_theme_group_context,{Naked Nations – Tribe Hong Kong}
theme_theme_group_context,"{Not Born Yesterday, Stamp Our Story, Dealer, The History of Indian Elections, Gendelyk, Our Land, Our Freedom, Ex You, Notes on a Summer, Bad Romance: The Vicky White Story, The Prince of Egypt: The Musical, Transformation, Amanita, Water Baby, The Prisoner of Wakhan, Please Look Up, Nyanga, Boots On The Ground, Two Balloons, Sorbeltza, 1984 at the BBC, 4 Walls, Feet Fear Freedom, Rencores, Girl, Paradise Is Burning, Zindagi Zindabaad, Psychedelic Revolution: The Secret History of the LSD Trade, Breath, Flowing - My Dream of More Freedom, Breaking

In [267]:
output_df, current_selected_df = compare_searches("What are some titles of dark comedy movies?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,"Night, Silence, Et Tu, Affirmations, Ab Absurdo, The Butler, End All, Say All, The Beast of Comedy, Dindin, Dead Serious, For When You Get Lost, The Roommate"
theme_answer,"Barely Breathing, Irish Catholic, End All, Say All, Dindin, Klokkenluider, Silence, Fishmonger, Død af grin, Do Not Expect Too Much from the End of the World, Natural Causes, Triple Oh!, Accessory, Sleepyhead, World's Greatest Dad, Deal of a Lifetime, Funny Not Funny, Mole, Stand-Up - My Life Is A Joke, Mike Ward : Noir, Sheryl, Death Perception, The Year of Staring at Noses, Affirmations, Eppura, Sweet Dreams, Sam Jay: Salute Me or Shoot Me, Abomination, Thiraiyin Marupakkam, Double Trouble, Anemone: A Fairy Tale for No Kids, Otto Baxter: Not A F***ing Horror Story, Should Be Long Gone, Holiday Heist, Au Revoir, Slumdogz, The Emu War, Home Town Down Time, The Moon Is Upside Down, Void, The Choice Has Been Made, Cannibal Mukbang, The Puppet Asylum, The Treasure of the Cyclops, Red Light District Shark Attack, The Happy Hotel, In Search Of...Detective Barnaby And His Descent Into The Haunted Mansion, Paranormal Drunktivity, Shaky Shivers, The Haunted Baby Carriage From Hell, Dare!, Ghost Babe, Horny Teenagers Must Die!, Chona 2: La puerca asesina"
theme_group_answer,"Dindin, Stand-Up - My Life Is A Joke, World's Greatest Dad, Silence, Død af grin, Fishmonger, Barely Breathing, Klokkenluider, Mole, With Love, from AIDAN, Do Not Expect Too Much from the End of the World, Death Perception, Home Town Down Time, Mike Ward : Noir, Accessory, The Choice Has Been Made, Funny Not Funny, Deal of a Lifetime, Natural Causes, Eppura, The Moon Is Upside Down, I'm Not a Robot, Irish Catholic, Affirmations, The Year of Staring at Noses, Abomination, Sam Jay: Salute Me or Shoot Me, Anemone: A Fairy Tale for No Kids, Sweet Dreams, Au Revoir, Slumdogz, Holiday Heist, Should Be Long Gone, Honto ni Atta! Noroi no Video 105, Honto ni Atta! Noroi no Video 104, V/H/S/85, Tenants, Chona 2: La puerca asesina, Horny Teenagers Must Die!, Ghost Babe, Dare!, The Haunted Baby Carriage From Hell, Shaky Shivers, Paranormal Drunktivity, Pinoy Ghost Tales, In Search Of...Detective Barnaby And His Descent Into The Haunted Mansion, The Happy Hotel, Red Light District Shark Attack, Dean of the Dead Presents: Holiday Horrors, Honto ni Atta! Noroi no Video 103, Otto Baxter: Not A F***ing Horror Story"
movie_only_context,"{Movie (Choke), Egun, mother, son, Comedy!, The Darker The Shadow, The Brighter The Light, Night, The Roommate, DarkGame, Dead Serious, Stitches, What's Behind You, Tastes of Horror, Et Tu, To the Brink, Into the Darkness, Kill The Lights, What the Blood Moon Brings: Messiah of Evil, A New American Nightmare, The Beast of Comedy, American Comedy, In The Shadow Of The Horns, GrimTrax, The Day Is Long and Dark, The Martini Shot, Elegy - Director's Cut, Little Death, The Butler, My Movie, For When You Get Lost, The Terror of Tarheel State: Making The Dark Power, Date Night, Ab Absurdo, Darkling, Power Cut, The Electric Fleapit, Murder In The Theater, MrMrMILKMAN in Donnie Darko, Brave the Dark, The Critic, Pattaya Explicit, What Happens in the Dark, Her Dark Reflection}"
theme_only_context,"{The Puppet Asylum, The Treasure of the Cyclops, The Emu War, Cannibal Mukbang}"
theme_group_only_context,"{I'm Not a Robot, V/H/S/85, Honto ni Atta! Noroi no Video 104, Honto ni Atta! Noroi no Video 103, With Love, from AIDAN, Honto ni Atta! Noroi no Video 105, Lessons of Tolerance, Pinoy Ghost Tales, Los, Tenants, Dean of the Dead Presents: Holiday Horrors, The End of Voyage}"
movie_theme_context,{}
movie_theme_group_context,{}
theme_theme_group_context,"{Shaky Shivers, Ghost Babe, Sleepyhead, Abomination, Double Trouble, Klokkenluider, World's Greatest Dad, Au Revoir, Slumdogz, The Choice Has Been Made, Triple Oh!, Chona 2: La puerca asesina, Thiraiyin Marupakkam, Dare!, Should Be Long Gone, Red Light District Shark Attack, Home Town Down Time, Barely Breathing, Sheryl, Th

In [268]:
output_df, current_selected_df = compare_searches("What are some movies about artificial intelligence?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,"The Diary of Sisyphus, Synthesis, Original Short Film, The Conscious ""I"", Mad Scientists: A.I. Maxima, 3 Dialogues About the Future, The Creator, William Quail’s Pyramid, Fair Weather Friends, A.I. Perversion, Flash Wars - Autonomous Weapons, A.I. and the Future of Warfare, To Live and Die in AI, Blue Fire, Primacy, Ilya, AI Lobotomy, 512X512, Double iSmart, Cyborg Society, Love Machina, Human or AI? The future of beauty standards, 4°C, EDDY BAIR, T-130, The Perfect Human, Telephones 4 Eyes, The Electric Kiss, Literally No Place, Sci-fi Silence, Two Gracious Uncles Smooched to the Beat, To The Future: The Movie, Deep Fake Love, The Beast, Teri Baaton Mein Aisa Uljha Jiya, The Vivid Unknown, AIMEE: The Visitor, A.I. Made Me Do It: Ophelia, Cosmic Miniatures, Storyteller, H-151: Perfect Machine, Aire: Just Breathe, Intercept: A Century of Signals Intelligence, Death Cleaning, Impossible Decision"
theme_answer,"CCC, Lady, Primacy, You Are Human, and You Are Lover, Landless, Illusions of Movement, Pesudo, So Unreal, CAPTCHA, The Creator, The Female Side Effect, 3 Dialogues About the Future, Love Me, The Beast, i ai, Cyborg Society, Blue Fire, H.O.M.E., Lights Over Montgomery County, Two Gracious Uncles Smooched to the Beat, A.I. Perversion, Cybergrime, The Conscious ""I"", Ilya, Love Machina, Sucker, Mad Scientists: A.I. Maxima, Bell Jingler 2023"
theme_group_answer,"CCC, The Perfect Human, CAPTCHA, i ai, The Beast, So Unreal, The Female Side Effect, Mad Scientists: A.I. Maxima, Lady, Cyborg Society, Pesudo, Illusions of Movement, 3 Dialogues About the Future, Love Me, You Are Human, and You Are Lover, Primacy, The Creator, Ilya, Landless, The Electric Kiss, Flash Wars - Autonomous Weapons, A.I. and the Future of Warfare, Smog, EDDY BAIR, Whisper of a Butterfly, Synthesis, Agents, A.I. Made Me Do It: Ophelia, Exit Pangea, Artificial Intelligence, 2023, 4°C, Fair Weather Friends, AIMEE: The Visitor, AI History 1890-2090, To Live and Die in AI, Dotty and the Tower of Eternity, The AI Report, A Tree Once Grew Here, Ghost in the Shell: SAC_2045 The Last Human, Voice in My Head, With Love, from AIDAN, Cosmic Miniatures, Aire: Just Breathe, MOVE, Literally No Place, Human or AI? The future of beauty standards, Storyteller, AI Lobotomy, Cybergrime, A.I. Perversion, Two Gracious Uncles Smooched to the Beat, Lights Over Montgomery County, H.O.M.E., Blue Fire, JUST DESSERTS, The Conscious ""I"", Sucker, Motherboard, Mis-Alignment, Taeter Burger, 21.10.23"
movie_only_context,"{William Quail’s Pyramid, Intercept: A Century of Signals Intelligence, Double iSmart, Impossible Decision, T-130, Death Cleaning, To The Future: The Movie, The Vivid Unknown, The Diary of Sisyphus, 512X512, Sci-fi Silence, H-151: Perfect Machine, Telephones 4 Eyes, Teri Baaton Mein Aisa Uljha Jiya, Original Short Film, Deep Fake Love}"
theme_only_context,{Bell Jingler 2023}
theme_group_only_context,"{Voice in My Head, Dotty and the Tower of Eternity, Mis-Alignment, Motherboard, Whisper of a Butterfly, Taeter Burger, Smog, Ghost in the Shell: SAC_2045 The Last Human, Agents, A Tree Once Grew Here, Exit Pangea, JUST DESSERTS, MOVE, 21.10.23}"
movie_theme_context,{Love Machina}
movie_theme_group_context,"{Artificial Intelligence, 2023, Human or AI? The future of beauty standards, Aire: Just Breathe, Synthesis, Storyteller, AI Lobotomy, The Perfect Human, The Electric Kiss, Fair Weather Friends, With Love, from AIDAN, Cosmic Miniatures, 4°C, Literally No Place, Flash Wars - Autonomous Weapons, A.I. and the Future of Warfare, A.I. Made Me Do It: Ophelia, EDDY BAIR, The AI Report, AIMEE: The Visitor, AI History 1890-2090, To Live and Die in AI}"
theme_theme_group_context,"{CAPTCHA, Lady, Landless, Lights Over Montgomery County, i ai, CCC, Pesudo, Sucker, H.O.M.E., You Are Human, and You Are Lover, The Female Side Effect, Love Me, Cybergrime}"
all_context,"{A.I. Perversion, The Conscious ""I"", The Beast, Primacy, Blue Fire, 3 Dialogues About the Futur

In [269]:
output_df, current_selected_df = compare_searches("What are some movies about growing up in a small town?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,"The Boyz, A Boy and a Girl, Letters Home, Lunar Oddities, Little Hollywood, The Tree Without Leaves, Into the Wild, Strawberry Fields Forever, Far From the Tree, Drugstore June, Golden City, The Garage, LOCALS, Rabbit Hole, Salman Society, Not Friends, Tyler, Skive, Life Inside the Seed, Ten Years, Ripples of Life, Happy Holla Day, Lost in the Night, Dasama, 8mm Boys, Five More Minutes, Woolly, Freaky Tales, Old Fox, Here and Nowhere, Pillow Talk, Lantrani, A Small Town Christmas, Happy Campers, Growing Up Green, The Boy from Lookout Mountain, The Old Man and The Land, Ein kleines Stück Kohle, Prisoners, Moving On, Old Times, Runaway"
theme_answer,"Smalltown Beast, That's How It Is, The Tree Without Leaves, Born Crying, The Yardley Boys, Lost in the Night, Sticky Fingers, The B in LGBT, The Pumpkin Man, Amandla, The Ridge Grave Girls"
theme_group_answer,"Sticky Fingers, The Yardley Boys, Warrior Strong, Hellcat, The B in LGBT, Takeshi, Amandla, Escape the Usual, Far From the Tree, Troublesome Creek, Lonely, A Harvest Homecoming, The Christmas Classic"
movie_only_context,"{Ein kleines Stück Kohle, The Boy from Lookout Mountain, Little Hollywood, Strawberry Fields Forever, Good Times, Tyler, Golden City, Here and Nowhere, Ten Years, Period, Dasama, A Small Town Christmas, Old Fox, Freaky Tales, Growing Up Green, Not Friends, Lantrani, The Old Man and The Land, A Boy and a Girl, Where the Tumbleweeds Grow, The Fell We Climb, 8mm Boys, Lunar Oddities, LOCALS, Little Death, Happy Holla Day, Prisoners, Skive, Into the Wild, Runaway, The Boyz, Salman Society, Pillow Talk, Woolly, Rabbit Hole, Drugstore June, Letters Home, Moving On, The Garage, Old Times, Happy Campers, The Godfrey Project, Life Inside the Seed, Five More Minutes}"
theme_only_context,"{Pet Sematary: Bloodlines, Öğretmen, Smalltown Beast, A Cowboy Christmas, Bizkarsoro, Kanni, Hometown Prison, Frogs, Meet Me at the Christmas Train Parade, Rickerl – Musik is höchstens a Hobby, Les més grans, Record Without Words, Samosa & Sons, Die Drei von der Müllabfuhr - Altlasten, Born Crying}"
theme_group_only_context,"{Fruit of the Poisonous Tree, Yaan Superstar, Pushing, Takeshi, Barkley Dubis' Danse Macabre, Troublesome Creek, Mavera, Nadhikalil Sundari Yamuna, Trueville: The Hole Story, Lonely, A Harvest Homecoming, Who's Yer Father?, Pimpi, Escape the Usual, Mark of the Werebeard, The Great Indian Family, Kahi De Ne Prem Chhe}"
movie_theme_context,"{Lost in the Night, That's How It Is, The Tree Without Leaves}"
movie_theme_group_context,{Far From the Tree}
theme_theme_group_context,"{The Pumpkin Man, Cooper Studios' Scream, Yuletide the Knot, HockeyTown Hoedown, Amandla, A Gluten-Free Christmas, The Yardley Boys, Field Day, Torn Heaven, Wild Woman, Christmas with a Kiss, ¿Quién es Keili González?, The White Crows, Adire, At The Roadhouse, Axeman at Cutters Creek 2, CHUM!, Holidays, River of Ghosts, The B in LGBT, The Perfect In-Laws, Mystery on Mistletoe Lane, La Suprema, Where the Cold Wind Blows, Squealer, Youth Haunted: Return of the Phantom Driver, A Town Called Love, Hellcat, The Ridge Grave Girls, Red Right Hand, The Christmas Classic, Me & Her, Whisky Business, Time for Her to Come Home for Christmas, The Naughty List, Zombie Town, To All a Good Night, Hotel Gran Victoria, Warrior Strong, Hello Stranger}"
all_context,"{Sticky Fingers, Ripples of Life}"


In [270]:
output_df, current_selected_df = compare_searches("What are some movies about Thanksgiving dinner?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,"Thanksgiving, Giving Thanks, Turkey Trouble, A Thanksgiving Christmas, Thanksgiving Roast 2, When the Trash Man Knocks, How to Ruin the Holidays"
theme_answer,"Giving Thanks, Thanksgiving Roast 2, A Thanksgiving Christmas, Turkey Trouble"
theme_group_answer,"When the Trash Man Knocks, Giving Thanks, Thanksgiving, Holiday Boyfriend, Thanksgiving Roast 2, A Thanksgiving Christmas"
movie_only_context,"{How to Ruin the Holidays, Family Ornaments, Istanbul Trilogy: Meze, The Poker Night, The Last Dinner, El banquete, The Departure, Holiday Road, Holiday Heist, The Last Supper, A Home for the Holidays, Dindin, Christmas with the Foxes, A Christmas Movie, Carmen & Moony, Dinner Time, A restaurant, a mouse, a dead man, a very particular family and a rainy dark night..., Renters: The Holiday Special, Happy Holla Day, Stir, The Ravioli Movie, The Amityville Whorror, The Hoop Dreams Holiday Special, The Holdovers, Momma Said Come Home For Christmas, The Role, The Perfect Christmas, Family Photo, Moving, Tomb Sweeping, Picking Mangoes, Going Home, Titanic, Suitable Version for Iranian Families, Whirlwind, El sabor de la Navidad, A Perfect Christmas Carol, Whatever It Takes, A Gettysburg Christmas, Table for Six 2, Christmas with Jerks, Pun Intended}"
theme_only_context,"{The Holiday Proposal Plan, Barry Manilow's A Very Barry Christmas, Dealing with Christmas, 'Twas the Text Before Christmas, Country Hearts Christmas, The Unofficial Science of Die Hard, Trouble with Johnny, I Love Nancy Meyers, A Biltmore Christmas}"
theme_group_only_context,"{Pokémon Original Short Animation: Homecoming, The Many Worlds of George Goodman, Guess Who, Noël Joyeux, Month of Madhu, Only Love Matters, My Big Fat Greek Wedding 3, Such a Lovely Day, André Rieu's White Christmas, Haul Out the Holly: Lit Up, Honeybun, No Love Lost, Never Been Chris'd, On the Shore, Locked Up, Marimacha, A Very Demi Holiday Special, Animal Crossing Christmas Festival: The Movie!, Lord Mayor's Christmas Carols, A Christmas for the Ages, Wide Closed, A Day and a Half, Christmas at Carbell Family Farm, Blood Actually: A Murder, They Hope Mystery, Holidays, Snot and Splash, The Ghost Lullaby, Nigella’s Amsterdam Christmas, Million to One, Big Brood, Leave, Thus Love Me, Enough for Now, Friends & Family Christmas, Mystic Christmas, Did You See The Hole That Mom Dig?, The Wasp and the Orchid, The Hatchling, Legend of El Cucuy, Ambrosia, A Spectacle, Heat Wave, Keys to the Heart, Figs, Band Four, Letters to Santa, Life and Death of a Christmas Tree}"
movie_theme_context,"{Just Like a Christmas Movie, Turkey Trouble}"
movie_theme_group_context,"{Sunday Dinner, Christmas on Cherry Lane, Family Time}"
theme_theme_group_context,"{A Little Gay, Ein Regenbogen zu Weihnachten, Los Mosquitos}"
all_context,"{Thanksgiving, A Thanksgiving Christmas, Holiday Boyfriend, When the Trash Man Knocks, The Great Turkey Town Miracle, Myristica, Giving Thanks, Thanksgiving Roast 2}"


In [278]:
output_df, current_selected_df = compare_searches("What are some movies about Europe in the 1960s?")
selected_df = pd.concat([selected_df, current_selected_df])
output_df

,result
movie_answer,Yesterday | Sea-64 | Dream Time | Pioniere der Filmmusik - Europas Sound für Hollywood | All Men Become Brothers | Finally Dawn | Fasolákia | Europa | The European Dream | Motherland | Keep Looking | From 8 Movies To 1 | Here | Celluloid Underground
theme_answer,The Eye of Spring | Oh Sweetie! | All That Is Sacred | 30 Kilometres per Second | Der Funker | Jokes & Cigarettes | Sea-64
theme_group_answer,Lex Barker - Westernheld und Playboy
movie_only_context,"{Dream Time, The Earthly Paradise, Romeo 1980, It's a Zabriskie, Zabriskie, Zabriskie, Zabriskie Point, Celluloid Underground, The Belgian Wave, All Men Become Brothers, The Movie Man, More Than Strangers, À la Recherche, The Great Glitch / Children of Paradise, Speak Sunlight, Storm Children, Keep Looking, Finally Dawn, Cinema's First Nasty Women, Manhattan Zodiac '77, Somewhere In Time, Scala!!!, Just Like in the Movies, La Reine, I’m Not Everything I Want To Be, ""WORDS"", Nowhere, Solitude, It's That, Motherland, Queeroslavia, The Male Gaze: A Better Tomorrow, Pink Triangles, Homosexuals Facing Nazism, From 8 Movies To 1, Fasolákia, Paris-Bruxelles, Here, Ecumenopolis, Casablanca, Soundtrack to a Coup d'État, Un'altra Italia era possibile - Il cinema di Giuseppe De Santis, Heiresses, yours,, Escape to Berlin, The European Dream, Surf on, Europe!}"
theme_only_context,"{Pressure Makes Diamonds, The Man from Nowhere, Я тебя люблю — Je t’aime, Spede, The Movie Emperor, Oh Sweetie!, Burnt Flowers, Housewife of the Year, Holy Family, 30 Kilometres per Second, The Red Vanity Case, All That Is Sacred, Child No. 182, Psychedelicized: The Electric Circus Story, Richland Descending, The Eye of Spring, The Archies, Anibal's Paradise, Onódy, a hatvanas évek császára, He Went That Way, Der Funker, Cart Return, This Is Ours, The First Women, Photosynthesis}"
theme_group_only_context,"{The House That Stood, El aprendiz, Légua, Agent of Solitude, Ancella d'Amore, Francisco Martins Rodrigues: Revolução ou Morte, Migrant Sea, Mothership, A Royal Christmas Holiday, The Walk, Seaside Special, Adriana Varejão: Between Flesh and Oceans, Tale of the Three Flames, 31 Ottobre, Para Norte, Walls, Chasing the Dazzling Light, Many Moons, Sounds of Codfish, Irreversível, 800 giorni, Her Name Was Gisberta, SAUDADE, Un hipster en la España vacía, Pepi Fandango, Nous n'avons pas peur des ruines, Alma Anciana, Dormouse Detectives, Made in Europe, To Be at Home, PosSession of the Spirit, Amelia’s Children, A Tomato Tragedy, From the Observatory they saw me, Clandestina, Dolce Far Niente (Sweet Doing Nothing), Lex Barker - Westernheld und Playboy, Fioretta, Goodbye First Love, The Permanent Picture, After Everything, Nativity, Tales of Oblivion}"
movie_theme_context,"{Lady with Lipstick, Yesterday}"
movie_theme_group_context,"{Europa, Runaway brains, Pioniere der Filmmusik - Europas Sound für Hollywood, Europe since Maastricht - A Journey through the Continent, to open a window}"
theme_theme_group_context,{Jokes & Cigarettes}
all_context,{Sea-64}


In [272]:
selected_df

,title,overview,query,retriever
0,Pioniere der Filmmusik - Europas Sound für Hollywood,"Hollywood film music has its roots in Europe. Three composers who fled war and National Socialism to the USA created the sound that still shapes film music today: Erich Wolfgang Korngold, Max Steiner and Franz Waxman. In the early 20th century, these classically trained composers transformed the methods acquired in Vienna and Berlin into a new American art form: film music. They balanced the relationship between image and sound and developed techniques and dramaturgical tricks to achieve the greatest possible effect on the viewer. Their influence is visible in the work of contemporary US composers such as John Williams and Jerry Goldsmith. Today, Oscar winner Hans Zimmer, Ramin Djawadi and Harold Faltermeyer continue this tradition. Their melodies are part of humanity's collective memory and reflect the combined traditions of European and American musical history. The documentary accompanies composers in their work and explores the European roots of Hollywood.",What are some titles of movies about classical music?,movie
1,Lenechka,"Leonid Desyatnikov is the most played Russian composer of today. Unparalleled, his music is performed all over the world from the Bolshoi Theatre and Paris’s Opéra Garnier to Milan’s La Scala and New York’s Carnegie Hall. While he usually shuns publicity, this time, he will open up to share some of his innermost thoughts. From the genius’s weekday routine and musings to the bold premiere involving the iconic conductor Teodor Currentzis, this movie is an attempt at unraveling the enigma that is the paradox of the composer’s music and persona.",What are some titles of movies about classical music?,movie
2,Symphonic Sketches,“Symphonic Sketches” tells the story of one classical music concert performed by the Denver Philharmonic Orchestra. Musicians from diverse racial backgrounds connect to the repertoire in unique ways and stage an unforgettable performance.,What are some titles of movies about classical music?,movie
3,A Piano Tune,"In a world imbued with magical realism, a few musical notes are enough to reunite those who love each other. Gabo, a man swept up in the whirlwind of war, dreams of this piano tune that will bring him home. With its use of magical realism and family love themes, 'A Piano Tune' will embark you into a life's journey in time, full of tense moments and pure emotion.\r The film is also a tribute to author Gabriel García Márquez, an emblematic figure of Colombian culture.",What are some titles of movies about classical music?,movie
4,Symphony of Suffering,"Symphony of Suffering is a showcase of shorts from some of the most prominent filmmakers within the fetish-gore sub-genre. An anthology with films from Jonathan Doe (Erotic Grotesque Nonsense trilogy), Jack Mulvanerty (Scarlet Piss Princess), White Gardenia (Mutilation Theater), and Blucifer (Putrid Menstrual Slime). With the feature scored to the Hurrian Hymn to Nikkal, the oldest known melody in recorded history; Symphony of Suffering attempts to showcase that the intertwining of pain and art are just as deeply infused and rooted in human expression as the origin of music itself.",What are some titles of movies about classical music?,movie
...,...,...,...,...
7,He Went That Way,"Set in 1964, a three-day journey along Route 66 begins when Bobby Falls, a 19-year-old serial killer, hitches a ride with Jim Goodwin, a celebrity animal handler travelling with his precious cargo: his TV chimpanzee, Spanky.",What are some movies about Europe in the 1960s?,theme
8,Burnt Flowers,"Following a visit from a mysterious femme fatale, detective Francis Alban finds herself embroiled in a missing persons case in which all is not what it seems. The trail leads her to a 1960s serial killer who was never caught and a violent gangster. As Detective Alban delves deeper into the criminal underbelly of Swinging Sixties London, will she uncover more about herself 

In [274]:
selected_df.to_csv("movie_answers.csv", index=False, header=True)

In [8]:
movie_query = """
MATCH (node)
RETURN node.overview AS text, score, node{.title, .tagline} AS metadata
"""

movie_vectorstore = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key),
    index_name="movie_text_vectors",
    retrieval_query=movie_query,
    url=neo4j_uri,
    username=neo4j_user,
    password=neo4j_password
)

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a movie expert providing information about movies.
                  Be as helpful as possible and return as much information as possible.
                  Do not answer any questions that do not relate to movies.
                  Always include the title of the movie in your answer.
                  Answer any questions based solely on the context below:
                  <context>
                  {context}
                  </context>"""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("user", "{input}"),
])

In [10]:
movie_info_template= "overview: {page_content}, title: {title}"
document_prompt = PromptTemplate(input_variables=["page_content", "title"],
                                 template=movie_info_template
                                 )


In [35]:
theme_group_retriever = theme_group_vectorstore.as_retriever(search_kwargs={"k": 5})
theme_retriever = theme_vectorstore.as_retriever(search_kwargs={"k":15})
movie_retriever = movie_vectorstore.as_retriever(search_kwargs={"k": 30})

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

combine_docs_chain = create_stuff_documents_chain(
    ChatOpenAI(temperature=0, openai_api_key=openai_api_key), prompt, document_prompt=document_prompt

)
theme_group_chain = create_retrieval_chain(theme_group_retriever, combine_docs_chain)
theme_chain = create_retrieval_chain(theme_retriever, combine_docs_chain)
movie_chain = create_retrieval_chain(movie_retriever, combine_docs_chain)

In [40]:
theme_group_retriever.get_relevant_documents("sharks")

[Document(page_content='The curse of Count Dracula lives on in shark-infested waters, claiming the lives of a tourist community. A sea hunt for the new species results in monsters, madness and bloodshed. This great white is putting the bite back into terror, and it has help with the aid of new vampires intent on seeing it survive.', metadata={'title': 'Sharkula'}),
 Document(page_content="Camille cultivates flowers on 7000 meters square in Aveyron, France. During all year, « le Jardin de Veillac » structures itself : building a new greenhouse, preparing seedlings, collecting flowers, preparing bouquets and, finally, selling them on Rodez's marketplace, where flowers unveil their beauty and spread in the city.", metadata={'title': 'La Ferme florale de Camille'}),
 Document(page_content='During the seed harvest season in Ban Rabue Village, Ngern is responsible for collecting and storing seeds for the upcoming year. Unfortunately, a group of black-faced thieves attempt to steal the seeds,

In [36]:
theme_group_chain.invoke({"input": "Find as many movies as you can about sharks."})

{'input': 'Find as many movies as you can about sharks.',
 'context': [Document(page_content="V is a writer who doesn't write, P is an actor who doesn't act, so to fill the void they go and see old movies and tell each other it was better before.", metadata={'title': 'Cary & James'}),
  Document(page_content='Trouble with Johnny explores the recurrence of loop, tempo, and flicker. The project uses found footage as a method for emphasizing the cinematic collisions between audio and visual composition. Working within the margins of structural filmmaking, Trouble with Johnny creates a visceral experience of fragmented sounds and cyclical imagery.  Likewise, Trouble with Johnny functions as a parody of the mainstream movie industry. The source material derives from the 1980’s coming of age film, The Karate Kid. Throughout the story, the main antagonist, a bully who goes by the name “Johnny”, is revered by a dedicated clique of subservient underlings. Applied as an experimental technique, t

In [31]:
theme_chain.invoke({"input": "Find as many movies as you can about sharks."})

{'input': 'Find as many movies as you can about sharks.',
 'context': [Document(page_content="V is a writer who doesn't write, P is an actor who doesn't act, so to fill the void they go and see old movies and tell each other it was better before.", metadata={'title': 'Cary & James'}),
  Document(page_content='Trouble with Johnny explores the recurrence of loop, tempo, and flicker. The project uses found footage as a method for emphasizing the cinematic collisions between audio and visual composition. Working within the margins of structural filmmaking, Trouble with Johnny creates a visceral experience of fragmented sounds and cyclical imagery.  Likewise, Trouble with Johnny functions as a parody of the mainstream movie industry. The source material derives from the 1980’s coming of age film, The Karate Kid. Throughout the story, the main antagonist, a bully who goes by the name “Johnny”, is revered by a dedicated clique of subservient underlings. Applied as an experimental technique, t

In [18]:
movie_chain.invoke({"input": "Find as many movies as you can about sharks."})

{'input': 'Find as many movies as you can about sharks.',
 'context': [Document(page_content="The ultimate deep dive into the world of shark cinema: filmmakers, critics, scholars and conservationists explore the weird, wild cinematic legacy of sharks on film and audiences' undying fascination with these misunderstood creatures.", metadata={'title': 'Sharksploitation'}),
  Document(page_content='In a not so distant future, great white sharks have gone extinct. Over fishing, climate change, pollution have all contributed to the species complete annihilation from the seas... but, land may be their new hunting ground! When a mad scientist plays god and genetically engineers a shark/human, he unleashes the highly viral monster on the unsuspecting citizens of a nearby small town. Now, a group of friends must band together if they are to survive the “sharkocalypse”. Run, hide, fight... or be turned into CHUM!', metadata={'title': 'CHUM!', 'tagline': 'You’re Gonna Need A Better Vaccine'}),
  D

In [37]:
theme_group_chain.invoke({"input": "Find as many movies as you can about high school."})

{'input': 'Find as many movies as you can about high school.',
 'context': [Document(page_content="V is a writer who doesn't write, P is an actor who doesn't act, so to fill the void they go and see old movies and tell each other it was better before.", metadata={'title': 'Cary & James'}),
  Document(page_content='Trouble with Johnny explores the recurrence of loop, tempo, and flicker. The project uses found footage as a method for emphasizing the cinematic collisions between audio and visual composition. Working within the margins of structural filmmaking, Trouble with Johnny creates a visceral experience of fragmented sounds and cyclical imagery.  Likewise, Trouble with Johnny functions as a parody of the mainstream movie industry. The source material derives from the 1980’s coming of age film, The Karate Kid. Throughout the story, the main antagonist, a bully who goes by the name “Johnny”, is revered by a dedicated clique of subservient underlings. Applied as an experimental techniq

In [20]:
summary_chain.invoke({"input": "Find as many movies as you can about high school."})

{'input': 'Find as many movies as you can about high school.',
 'context': [Document(page_content="V is a writer who doesn't write, P is an actor who doesn't act, so to fill the void they go and see old movies and tell each other it was better before.", metadata={'title': 'Cary & James'}),
  Document(page_content='Trouble with Johnny explores the recurrence of loop, tempo, and flicker. The project uses found footage as a method for emphasizing the cinematic collisions between audio and visual composition. Working within the margins of structural filmmaking, Trouble with Johnny creates a visceral experience of fragmented sounds and cyclical imagery.  Likewise, Trouble with Johnny functions as a parody of the mainstream movie industry. The source material derives from the 1980’s coming of age film, The Karate Kid. Throughout the story, the main antagonist, a bully who goes by the name “Johnny”, is revered by a dedicated clique of subservient underlings. Applied as an experimental techniq